# Model Evaluation

Load packages

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import seaborn as sns

In [ ]:
from sklearn.metrics import confusion_matrix
from keras import backend as K

Load Model

In [ ]:
model = tf.keras.models.load_model('/tmp/ml-pipeline/model/digit-recognition.keras')

Load data

In [ ]:
X_val = np.load('/tmp/ml-pipeline/data/X_val.npy')
y_val = np.load('/tmp/ml-pipeline/data/y_val.npy')
X_test = np.load('/tmp/ml-pipeline/data/X_test.npy')

Define metrics

In [ ]:
# Precision (using keras backend)
def precision_metric(y_true, y_pred):
    threshold = 0.5  # Training threshold 0.5
    y_pred_y = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold), K.floatx())

    true_positives = K.sum(K.clip(y_true * y_pred, 0, 1))
    false_negatives = K.sum(K.clip(y_true * (1-y_pred), 0, 1))
    false_positives = K.sum(K.clip((1-y_true) * y_pred, 0, 1))
    true_negatives = K.sum(K.clip((1 - y_true) * (1-y_pred), 0, 1))

    precision = true_positives / (true_positives + false_positives + K.epsilon())
    return precision

# Recall (using keras backend)
def recall_metric(y_true, y_pred):
    threshold = 0.5 #Training threshold 0.5
    y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), threshold), K.floatx())

    true_positives = K.sum(K.clip(y_true * y_pred, 0, 1))
    false_negatives = K.sum(K.clip(y_true * (1-y_pred), 0, 1))
    false_positives = K.sum(K.clip((1-y_true) * y_pred, 0, 1))
    true_negatives = K.sum(K.clip((1 - y_true) * (1-y_pred), 0, 1))

    recall = true_positives / (true_positives + false_negatives + K.epsilon())
    return recall

# F1-score (using keras backend)
def f1_metric(y_true, y_pred):
    precision = precision_metric(y_true, y_pred)
    recall = recall_metric(y_true, y_pred)
    f1 = 2 * ((precision * recall) / (recall+precision+K.epsilon()))
    return f1


Predict

In [ ]:
y_val_pred = np.argmax(model.predict(X_val), axis=1)
y_val_pred

In [ ]:
y_val_true = np.argmax(y_val,axis=1)
y_val_true

Confusion matrix

In [ ]:
cm = confusion_matrix(np.argmax(y_val,axis=1), y_val_pred)

In [ ]:
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion matrix')
plt.xlabel('Pred')
plt.ylabel('True')

Compile metrics and define artifacts

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
acc = accuracy_score(y_val_true, y_val_pred)
f1_macro = f1_score(y_val_true, y_val_pred, average="macro")
rec = recall_score(y_val_true, y_val_pred, average="macro")
prec = precision_score(y_val_true, y_val_pred, average="macro")
print(f'accuracy_score: {acc}')
print(f'f1_score_macro: {f1_macro}')
print(f'precision_score: {prec}')
print(f'recall_score: {rec}')

In [ ]:
import json

metadata = {
    'metrics': [
        {
            'name': 'accuracy-score',
            'numberValue':  acc,
            'format': 'PERCENTAGE'
        },
        {
            'name': 'f1-score',
            'numberValue':  f1_macro,
            'format': 'PERCENTAGE'
        },
        {
            'name': 'precision-score',
            'numberValue':  prec,
            'format': 'PERCENTAGE'
        },
        {
            'name': 'recall-score',
            'numberValue':  rec,
            'format': 'PERCENTAGE'
        }
    ]
}
with open('./mlpipeline-metrics.json', 'w') as f:
    json.dump(metadata, f)

> <div class="alert alert-block alert-info">
<b>Ok, not bad for our first model! The model has done a pretty good job at classifying each class and is obtaining a high accuracy score. Now we should take a look at the examples the model is misclassifying.</b>
</div>

In [ ]:
y_test_pred = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred,axis=1)
test_results = pd.DataFrame({'ImageID': np.arange(1,28001,1), 'Label': y_test_pred})